In [8]:
import gradio as gr
import pandas as pd
import random

# Load quiz data
quiz_data = pd.read_csv("quiz_data.csv")

# Initialize user state
state = {
    "current_question": None,
    "score": 0,
    "questions_asked": 0,
    "difficulty": "Easy",  # Start with easy questions
}

# Function to select the next question based on difficulty
def get_next_question(difficulty):
    filtered_data = quiz_data[quiz_data["Difficulty"] == difficulty]
    question_row = filtered_data.sample(1).iloc[0]
    return {
        "question": question_row["Question"],
        "options": [
            question_row["Option1"],
            question_row["Option2"],
            question_row["Option3"],
            question_row["Option4"],
        ],
        "answer": question_row["Answer"],
    }

# Function to handle quiz logic
def quiz_bot(user_choice):
    global state

    # Check if it's the first question
    if state["current_question"] is None:
        # Get the first question
        question_data = get_next_question(state["difficulty"])
        state["current_question"] = question_data
        return question_data["question"], question_data["options"], "Make your choice!"

    # Check the user's answer
    correct_answer = state["current_question"]["answer"]
    feedback = ""
    if user_choice == correct_answer:
        state["score"] += 1
        feedback = "Correct!"
        # Increase difficulty after 2 correct answers
        if state["score"] % 2 == 0:
            state["difficulty"] = "Medium" if state["difficulty"] == "Easy" else "Hard"
    else:
        feedback = f"Wrong! The correct answer was {correct_answer}."
        # Reduce difficulty after a wrong answer
        if state["difficulty"] == "Hard":
            state["difficulty"] = "Medium"
        elif state["difficulty"] == "Medium":
            state["difficulty"] = "Easy"

    # Update question count
    state["questions_asked"] += 1

    # Get the next question
    question_data = get_next_question(state["difficulty"])
    state["current_question"] = question_data

    # Return results
    return (
        question_data["question"],
        question_data["options"],
        f"{feedback} Your current score is {state['score']}."
    )

# Gradio Interface
def interactive_quiz(choice=None):
    return quiz_bot(choice)

quiz_interface = gr.Interface(
    fn=interactive_quiz,
    inputs=gr.Radio(choices=["Start", "Option1", "Option2", "Option3", "Option4"], label="Select an Option"),
    outputs=[
        gr.Textbox(label="Question"),
        gr.Radio(label="Options", choices=["Option1", "Option2", "Option3", "Option4"]),
        gr.Textbox(label="Feedback"),
    ],
    live=True,
)

quiz_interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://112e616ea3daf181c0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
